# Setup

In [1]:
import os
from pathlib import Path
import sys

# If we're using Google Colab, we set the environment variable to point to the relevant folder in our Google Drive:
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    os.environ['SKIN_LESION_CLASSIFICATION'] = '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification'

# Otherwise, we use the environment variable on our local system:
project_environment_variable = "SKIN_LESION_CLASSIFICATION"

# Path to the root directory of the project:
project_path = Path(os.environ.get(project_environment_variable))

# Relative path to /scripts (from where custom modules will be imported):
scripts_path = project_path.joinpath("scripts")

# Add this path to sys.path so that Python will look there for modules:
sys.path.append(str(scripts_path))

# Now import path_step from our custom utils module to create a dictionary to all subdirectories in our root directory:
from utils import path_setup
path = path_setup.subfolders(project_path)

Mounted at /content/drive
path['project'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification
path['models'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models
path['notebooks'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/notebooks
path['scripts'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/scripts
path['images'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/images


<a id='model_setup'></a>
# Model setup

In [2]:
from typing import Type, Union      # For type hints
from processing import process      # Custom module for processing metadata

data_dir: Path = path["images"]     # Path to directory containing metadata.csv file
csv_filename: str = "metadata.csv"  # The filename

tvr: int = 3              # Ratio of training set to validation set. See discussion below for explanation.
seed: int = 0             # Random seed for parts of the process where randomness is called for.
keep_first: bool = False  # If False, then, for each lesion, we choose a random image to assign to our training set.
stratified: bool = True   # If True, we stratify classes so that the proportions remain as stable as possible after train/val split.
                          # If False, the proportions will be roughly similar.

to_classify: Union[list, dict] = ["mel",   # These are the lesion types we are interested in classifying.
                                  "bcc",   # Any missing ones will be grouped together as the 0-label class: no need to write "other" here.
                                  "akiec", # If 'other' is not desired, use restrict_to attribute above
                                  "nv",]   # Can also be a dictionary, like { 'malignant' : ['mel', 'bcc'], 'benign' : ['nv', 'bkl']}

train_one_img_per_lesion: Union[None, bool] = True

In [3]:
# Create an instance of the process class with attribute values as above.
base_t1 = process(data_dir=data_dir,
                  csv_filename=csv_filename,
                  tvr=tvr,
                  seed=seed,
                  keep_first=keep_first,
                  stratified=stratified,
                  to_classify=to_classify,
                  train_one_img_per_lesion=train_one_img_per_lesion,)

- Loaded file '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/images/metadata.csv'.
- Inserted 'num_images' column in dataframe, to the right of 'lesion_id' column.
- Inserted 'label' column in dataframe, to the right of 'dx' column: 
  {'bkl': 0, 'df': 0, 'vasc': 0, 'akiec': 1, 'bcc': 2, 'mel': 3, 'nv': 4}
- Added 'set' column to dataframe, with values 't1', 'v1', 'ta', and 'va', to the right of 'localization' column.
- Basic, overall dataframe (pre-train/test split): self.df
- Training set (not balanced, one image per lesion): self.df_train
- Validation set (not expanded, one image per lesion): self.df_val1
- Validation set (not expanded, use all images of each lesion): self.df_val_a
- Small sample dataframes for code testing: self._df_train_code_test, self._df_val1_code_test, self._df_val_a_code_test


In [4]:
import torchvision.transforms as transforms
transform = transforms.Compose([
transforms.Resize((224,224)), # Resize images to fit ResNet input size
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

In [5]:
import pandas as pd
from typing import Union, List, Callable
import torchvision.models as models

source: Union[process, pd.DataFrame] = base_t1        # Processed data to be fed into model for training.
                                                      # Must either be an instance of the process class, or a dataframe of the same format as source.df if source were an instance of the process class.
model_dir: Path = path["models"]                      # Path to directory where models/model info/model results are stored.

transform: Union[None,
                 transforms.Compose,
                 List[Callable]] = transform     # Transform to be applied to images before feeding into neural network.

filename_stem: Union[None, str] = "rn18"         # For saving model and related files. Default "rn18" (if ResNet model) or "EffNet" (if EfficientNet), or "cnn".
filename_suffix: Union[None, str] = "base" # Something descriptive and unique for future reference. Default empty string "".

# model: Union[None, models.ResNet, models.EfficientNet] = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT) # Pre-trained model. Default: ResNet18.
model: Union[None, models.ResNet, models.EfficientNet] = models.resnet18(weights="ResNet18_Weights.DEFAULT")
unfreeze_all: Union[None, bool] = False
# overwrite: Union[None, bool] = True

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 128MB/s]


In [6]:
# Create an instance of the resnet18 class with attribute values as above.
from multiclass_models import cnn

rn18_base_t1 = cnn(source=source,
                    model_dir=model_dir,
                    transform=transform,
                    filename_stem=filename_stem,
                    filename_suffix=filename_suffix,
                    model=model,
                    unfreeze_all=unfreeze_all,)

New files will be created. 
Base filename: rn18_t1_uflast_10e_base_00
Attributes saved to file: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_t1_uflast_10e_base_00_attributes.json


In [8]:
import time

tic = time.time()
rn18_base_t1.train()
toc = time.time()
print(f"Elapsed time: {toc - tic}")

Batch: 0, running loss: 1.7447086572647095
Batch: 10, running loss: 12.447573840618134
Batch: 20, running loss: 20.117386043071747
Batch: 30, running loss: 26.97942692041397
Batch: 40, running loss: 34.38532626628876
Batch: 50, running loss: 40.76688960194588
Batch: 60, running loss: 46.584028244018555
Batch: 70, running loss: 52.72929385304451
Batch: 80, running loss: 58.09044283628464
Batch: 90, running loss: 64.66766738891602
Batch: 100, running loss: 70.80188885331154
Batch: 110, running loss: 76.77182164788246
Batch: 120, running loss: 83.6437654197216
Batch: 130, running loss: 90.03966680169106
Batch: 140, running loss: 96.80034276843071
Batch: 150, running loss: 103.19598236680031
Batch: 160, running loss: 110.00941130518913
Batch: 170, running loss: 117.44139078259468
Epoch 0, train_loss: 0.6841942672617733
Epoch 0, val1_loss: 0.03532804877071057
Epoch 0, val1_loss: 0.07172599889464297
Epoch 0, val1_loss: 0.10818138769117452
Epoch 0, val1_loss: 0.13700646465107547
Epoch 0, val1

In [9]:
# import torch
# import torch.nn as nn

# instance = rn18_base_t1

# model = models.resnet18()
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, len(instance.label_codes))

# file_path_pth = instance.model_dir.joinpath(instance._filename + ".pth")
# state_dict = torch.load(file_path_pth)
# model.load_state_dict(state_dict)

# # model = models.efficientnet_b0()
# # num_ftrs = model.classifier[1].in_features
# # model.classifier[1] = nn.Linear(num_ftrs, len(instance.label_codes))

# instance.model = model
# instance.state_dict = state_dict

In [15]:
# from utils import print_header
from multiclass_models import get_probabilities

instance = rn18_base_t1

tic = time.time()

instance.df_probabilities_val1 = get_probabilities(df=instance.df_val1,
                                                   data_dir=instance.data_dir,
                                                   model_dir=instance.model_dir,
                                                   model=instance.model,
                                                   filename=instance._filename,
                                                   label_codes=instance.label_codes,
                                                   transform=instance.transform,
                                                   batch_size=instance.batch_size,
                                                   Print=False,
                                                   save_as=instance._filename + "_val1",)

toc = time.time()

print(f"Elapsed time: {toc - tic}")

instance.df_probabilities_val_a = get_probabilities(df=instance.df_val_a,
                                                    data_dir=instance.data_dir,
                                                    model_dir=instance.model_dir,
                                                    model=instance.model,
                                                    filename=instance._filename,
                                                    label_codes=instance.label_codes,
                                                    transform=instance.transform,
                                                    batch_size=instance.batch_size,
                                                    Print=False,
                                                    save_as=instance._filename + "_val_a",)
tic = time.time()

print(f"Elapsed time: {tic - toc}")

Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_t1_uflast_10e_base_00_val1_probabilities.csv
Elapsed time: 29.39939785003662
Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_t1_uflast_10e_base_00_val_a_probabilities.csv
Elapsed time: 38.96911430358887


In [ ]:
# instance = rn18_base_t1

# file_path1 = instance.model_dir.joinpath("rn18_t1_uflast_10e_base_00_val1_probabilities.csv")
# file_path_a = instance.model_dir.joinpath("rn18_t1_uflast_10e_base_00_val_a_probabilities.csv")

# instance.df_probabilities_val1 = pd.read_csv(file_path1, index_col=0)
# instance.df_probabilities_val_a = pd.read_csv(file_path_a, index_col=0)

In [17]:
from utils import print_header

instance = rn18_base_t1

print_header("Validation set: one image per lesion")
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')]
display(instance.df_probabilities_val1[display_columns].head())

print_header("Validation set: all images per lesion")
display(instance.df_probabilities_val_a[display_columns].head())


VALIDATION SET: ONE IMAGE PER LESION



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv
0,HAM_0002730,ISIC_0025661,bkl,0.050186,0.551444,0.383338,0.008397,0.006635
1,HAM_0001466,ISIC_0027850,bkl,0.561856,0.219038,0.027982,0.184205,0.006919
2,HAM_0002761,ISIC_0029068,bkl,0.104806,0.117343,0.645730,0.003072,0.129048
3,HAM_0004234,ISIC_0029396,bkl,0.047635,0.064842,0.153847,0.063390,0.670287
4,HAM_0001949,ISIC_0025767,bkl,0.951744,0.000080,0.000097,0.004153,0.043926



VALIDATION SET: ALL IMAGES PER LESION



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv
0,HAM_0002730,ISIC_0026769,bkl,0.013689,0.680603,0.304388,0.000793,0.000527
1,HAM_0002730,ISIC_0025661,bkl,0.050186,0.551444,0.383338,0.008397,0.006635
2,HAM_0001466,ISIC_0031633,bkl,0.719853,0.159101,0.029969,0.081175,0.009901
3,HAM_0001466,ISIC_0027850,bkl,0.561856,0.219038,0.027982,0.184205,0.006919
4,HAM_0002761,ISIC_0029176,bkl,0.159595,0.349277,0.472283,0.002441,0.016404


In [19]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = rn18_base_t1

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = None#{ 'max' : ['mel', 'bcc', 'akiec'], 'min' : ['nv'], 'mean' : ['other']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = None# OrderedDict([('mel',0.4), ('bcc', 0.4), ('akiec', 0.4)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = None#OrderedDict([('nv',0.6)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = None #OrderedDict([('mel',1), ('bcc',1), ('akiec',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')] + ['pred', 'pred_final']
display(instance.df_pred_val1[display_columns].head())

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a[display_columns].head())


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0025661,bkl,0.050186,0.551444,0.383338,0.008397,0.006635,1,1
1,HAM_0001466,ISIC_0027850,bkl,0.561856,0.219038,0.027982,0.184205,0.006919,0,0
2,HAM_0002761,ISIC_0029068,bkl,0.104806,0.117343,0.645730,0.003072,0.129048,2,2
3,HAM_0004234,ISIC_0029396,bkl,0.047635,0.064842,0.153847,0.063390,0.670287,4,4
4,HAM_0001949,ISIC_0025767,bkl,0.951744,0.000080,0.000097,0.004153,0.043926,0,0



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0026769,bkl,0.013689,0.680603,0.304388,0.000793,0.000527,1,1
1,HAM_0002730,ISIC_0025661,bkl,0.050186,0.551444,0.383338,0.008397,0.006635,1,1
2,HAM_0001466,ISIC_0031633,bkl,0.719853,0.159101,0.029969,0.081175,0.009901,0,0
3,HAM_0001466,ISIC_0027850,bkl,0.561856,0.219038,0.027982,0.184205,0.006919,0,0
4,HAM_0002761,ISIC_0029176,bkl,0.159595,0.349277,0.472283,0.002441,0.016404,2,2


In [20]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = rn18_base_t1
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 2
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,170,8,3,8,36,225,0.755556
akiec,29,19,3,0,6,57,0.333333
bcc,22,23,19,3,15,82,0.231707
mel,63,1,2,42,46,154,0.272727
nv,111,3,3,22,"1,212","1,351",0.897113
All,395,54,30,75,"1,315","1,869",_
precision,0.43038,0.351852,0.633333,0.56,0.921673,_,0.354991



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,177,5,2,6,35,225,0.786667
akiec,33,15,4,0,5,57,0.263158
bcc,31,21,18,2,10,82,0.219512
mel,75,0,1,38,40,154,0.246753
nv,117,4,3,26,"1,201","1,351",0.888971
All,433,45,28,72,"1,291","1,869",_
precision,0.408776,0.333333,0.642857,0.527778,0.930287,_,0.32138


In [21]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = rn18_base_t1

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 2
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.782236,0.498087,NaN,NaN,0.533681,0.501211,0.492888,0.53421,0.921955,0.925413,0.860778



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.775281,0.481012,NaN,NaN,0.516836,0.480965,0.47291,0.529302,0.921939,0.927203,0.859617


In [22]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = rn18_base_t1

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = { 'max' : ['mel', 'bcc', 'akiec'], 'min' : ['nv'], 'mean' : ['other']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = OrderedDict([('mel',0.4), ('bcc', 0.4), ('akiec', 0.4)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = None#OrderedDict([('nv',0.6)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = None #OrderedDict([('mel',1), ('bcc',1), ('akiec',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')] + ['pred', 'pred_final']
display(instance.df_pred_val1[display_columns].head())

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a[display_columns].head())


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0025661,bkl,0.050186,0.551444,0.383338,0.008397,0.006635,1,1
1,HAM_0001466,ISIC_0027850,bkl,0.561856,0.219038,0.027982,0.184205,0.006919,0,0
2,HAM_0002761,ISIC_0029068,bkl,0.104806,0.117343,0.645730,0.003072,0.129048,2,2
3,HAM_0004234,ISIC_0029396,bkl,0.047635,0.064842,0.153847,0.063390,0.670287,4,4
4,HAM_0001949,ISIC_0025767,bkl,0.951744,0.000080,0.000097,0.004153,0.043926,0,0



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0026769,bkl,0.013689,0.680603,0.304388,0.000793,0.000527,1,1
1,HAM_0002730,ISIC_0025661,bkl,0.050186,0.551444,0.383338,0.008397,0.006635,1,1
2,HAM_0001466,ISIC_0031633,bkl,0.719853,0.159101,0.029969,0.081175,0.009901,0,0
3,HAM_0001466,ISIC_0027850,bkl,0.561856,0.219038,0.027982,0.184205,0.006919,0,0
4,HAM_0002761,ISIC_0029176,bkl,0.159595,0.349277,0.472283,0.002441,0.016404,2,2


In [23]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = rn18_base_t1
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 2
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,170,8,3,8,36,225,0.755556
akiec,28,19,4,0,6,57,0.333333
bcc,22,21,21,3,15,82,0.256098
mel,63,1,2,44,44,154,0.285714
nv,111,3,3,23,"1,211","1,351",0.896373
All,394,52,33,78,"1,312","1,869",_
precision,0.431472,0.365385,0.636364,0.564103,0.923018,_,0.365635



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,176,6,2,6,35,225,0.782222
akiec,32,15,5,0,5,57,0.263158
bcc,31,19,20,2,10,82,0.243902
mel,75,0,1,38,40,154,0.246753
nv,117,4,3,27,"1,200","1,351",0.888231
All,431,44,31,73,"1,290","1,869",_
precision,0.408353,0.340909,0.645161,0.520548,0.930233,_,0.32903


In [24]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = rn18_base_t1

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 2
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.783842,0.505415,NaN,NaN,0.542154,0.510385,0.501121,0.538664,0.921955,0.925413,0.860778



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.775281,0.484853,NaN,NaN,0.521273,0.486229,0.477506,0.529548,0.921939,0.927203,0.859617
